## Convert Kili to train format

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import shutil
import os

In [61]:
# Read csv
kili_df = pd.read_csv("/home/martinson/datasets_csv/kili_segmentation.csv")
conv_df = pd.read_csv("/home/martinson/datasets_csv/dataset_conversation.csv")

# Transform to convinient format and delete unnecessary
kili_df['source_path'] = kili_df['source_path'].apply(lambda x: x[3:])
conv_df['source_path'] = conv_df['source_path'].apply(lambda x: x[15:] if x[:15] == "D:\\measurements" else None)
conv_df = conv_df.dropna()

In [15]:
# Take all unique paths (these are different images)
source_path_unique = kili_df['source_path'].unique()

# Extract image names from conversation csv
image_names = []
for i in source_path_unique:
    image_names.append(list(conv_df[conv_df['source_path'] == i]['annotation_item_id'])[0])

In [29]:
# Make new DataFrame with real image name and source path from kili
df_img_ind = pd.DataFrame(columns=["image_name", "id"])

for i in range(len(image_names)):
    inds = kili_df[kili_df['source_path'] == source_path_unique[i]].index
    for j in inds:
        df_img_ind.loc[len(df_img_ind)] = [image_names[i], j]

In [22]:
# array for missed images
all_mis = []

# copy images from old folder to new one
for i in image_names:
    name_1 = "/home/martinson/data_2/img_seg_2/" + str(i) + ".png"
    name_2 = "/home/martinson/data_2/images/" + str(i) + ".png"
    try:
        shutil.copy(name_2, name_1)
    except:
        all_mis.append(i)

In [ ]:
# write missed images to file
with open("missed.txt", 'w') as f:
    for i in all_mis:
        f.write(str(i))
        f.write("\n")

In [6]:
# missed images was received and they are located in folder "missing"
# copy missed images 
for i in os.listdir("/home/martinson/data_2/missing"):
    a = i.split(".")
    name_2 = "/home/martinson/data_2/missing/" + i
    name_1 = "/home/martinson/data_2/images_segment/" + a[0] + ".png"
    shutil.copy(name_2, name_1)

In [57]:
# Make label files from kili
for i in source_path_unique:
    image_name = list(conv_df[conv_df['source_path'] == i]['annotation_item_id'])[0]
    file_name = "/home/martinson/data_2/labels_2/" + str(image_name) + ".txt"
    with open(file_name, 'w') as f:
        for j in kili_df[kili_df['source_path'] == i].values:
            label = 0
            if j[3] == 'SHARP_CAPILLARY':
                label = 1
            f.write(str(label))
            f.write(" ")
            x = j[6][1:-1].split(", ")
            y = j[7][1:-1].split(", ")
            for k in range(len(x)):
                f.write(x[k])
                f.write(" ")
                f.write(y[k])
                f.write(" ")
            f.write("\n")

### Split to train, test, val

In [13]:
from sklearn.model_selection import train_test_split

In [ ]:
# split
train, test = train_test_split(image_names, test_size=0.2, random_state=42)
val, test = train_test_split(test, test_size=0.5, random_state=42)

In [16]:
# to each txt file save image names
# save splitted results
with open("/home/martinson/data_2/test_seg.txt", "w") as f:
    for i in test:
        f.write("/home/martinson/data_2/images/" + str(i) + ".png" + "\n")

with open("/home/martinson/data_2/train_seg.txt", "w") as f:
    for i in train:
        f.write("/home/martinson/data_2/images/" + str(i) + ".png" + "\n")

with open("/home/martinson/data_2/val_seg.txt", "w") as f:
    for i in val:
        f.write("/home/martinson/data_2/images/" + str(i) + ".png" + "\n")

## Work with new Kili

Make same operations

In [5]:
# read csv
kili_new = pd.read_csv("/home/martinson/datasets_csv/df_kili_only_review.csv")
kili_new['source_path'] = kili_new['source_path'].apply(lambda x: x.split("/")[-1].split(".")[0])

In [8]:
# Take all unique paths (these are different images)
source_path_unique = kili_new['source_path'].unique()

In [17]:
# Make label files from kili
for i in source_path_unique:
    image_name = "/home/martinson/data_small/images/" + i + ".png"
    file_name = "/home/martinson/data_small/labels/" + i + ".txt"
    with open(file_name, 'w') as f:
        for j in kili_new[kili_new['source_path'] == i].values:
            label = [0]
            if j[4] == 'SHARP_CAPILLARY':
                label = [1]
            # interpret "INTERSECTION" as 2 records: SHARP and BLURRY
            if j[4] == 'INTERSECTION':
                label = [0, 1]
            x = j[8][1:-1].split(", ")
            y = j[9][1:-1].split(", ")
            for lb in label:
                f.write(str(lb))
                f.write(" ")
                for k in range(len(x)):
                    f.write(x[k])
                    f.write(" ")
                    f.write(y[k])
                    f.write(" ")
            f.write("\n")

In [14]:
# split for train and val
train, val = train_test_split(source_path_unique, test_size=0.1, random_state=42)

In [15]:
# to each txt file save image names
# save splitted results
with open("/home/martinson/data_small/train.txt", 'w') as f:
    for i in train:
        image_name = "/home/martinson/data_small/images/" + i + ".png"
        f.write(image_name)
        f.write("\n")

with open("/home/martinson/data_small/val.txt", 'w') as f:
    for i in val:
        image_name = "/home/martinson/data_small/images/" + i + ".png"
        f.write(image_name)
        f.write("\n")